In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.ensemble import ExtraTreesRegressor

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import *
from sklearn.inspection import permutation_importance
from statsmodels.stats.outliers_influence import variance_inflation_factor

import eli5
from eli5.sklearn import PermutationImportance

import warnings
warnings.filterwarnings("ignore")

# Calculate features

In [2]:
def compute_features(df):
    # intraday return features
    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17res'
    features = temp.reset_index()

    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # 17:30 ma raw return features
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = '17ma1raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = '17ma3raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = '17ma5raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = '17ma20raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    features['17ma1-3raw'] = features['17ma1raw'] - features['17ma3raw']
    features['17ma1-5raw'] = features['17ma1raw'] - features['17ma5raw']
    features['17ma1-20raw'] = features['17ma1raw'] - features['17ma20raw']

    # 17:30 ma residual return features
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = '17ma1res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = '17ma3res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = '17ma5res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = '17ma20res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    features['17ma1-3res'] = features['17ma1res'] - features['17ma3res']
    features['17ma1-5res'] = features['17ma1res'] - features['17ma5res']
    features['17ma1-20res'] = features['17ma1res'] - features['17ma20res']

    # market value
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id') * df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'market_value'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # intraday volume features
    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17vol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='16:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') - df[df['Time']=='10:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')) / df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17vol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # turnover ma
    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma1'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma3'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma5'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma20'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # original features 
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='estVol winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'estVol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid17'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid16'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid10'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'vol17'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # Normalization
    #features.iloc[:,2:] = (features.iloc[:,2:] - features.iloc[:,2:].mean())/features.iloc[:,2:].std()

    # Merging with target
    target = df[df["Time"] == "17:30:00.000"][['Date','Id','ResidualNoWinsorCumReturn winsorized']].copy()
    merged = features.merge(target ,on =['Date','Id']).drop_duplicates()
    merged = merged.sort_values(['Id','Date'],ascending = [True, True])
    merged["y"] = merged.groupby(['Id'])["ResidualNoWinsorCumReturn winsorized"].shift(-1)
    merged = merged.dropna(how = 'any')
    merged = merged.sort_values('Date',ascending=True)
    merged_with_dates = merged
    merged_with_dates.to_csv("merged_with_dates.csv")
    merged = merged.drop(["Id", "Date","ResidualNoWinsorCumReturn winsorized"], axis = 1)
    merged.to_csv('beforeTrainning.csv')
    return merged, merged_with_dates

In [3]:
df = pd.read_csv(r"C:\Users\Lenovo\Python workshop\project\merged_data_fillna_timeid.csv", sep = ",")
m, mwd = compute_features(df)
m

,10/17res,10/17raw,16/17res,16/17raw,17ma1raw,17ma3raw,17ma5raw,17ma20raw,17ma1-3raw,17ma1-5raw,...,turnover_ma1,turnover_ma3,turnover_ma5,turnover_ma20,estVol,cleanMid17,cleanMid16,cleanMid10,vol17,y
19,-0.014479,0.004383,0.003725,0.001532,-0.002551,-4.328447e-03,0.012672,0.001606,0.001777,-0.015223,...,5127.454439,6982.580835,11541.301587,10753.510582,0.296036,33.452305,33.503600,33.599230,872180.0,-0.017619
521779,0.006544,0.019172,0.001098,-0.002364,-0.013599,-4.222190e-03,-0.015091,-0.002486,-0.009377,0.001492,...,4172.375545,3466.191410,3757.401356,2936.059761,0.178838,17.535980,17.494583,17.875440,4175388.0,-0.000857
523534,0.007003,0.021363,0.002370,0.001177,-0.018284,-6.530792e-04,0.000158,-0.004395,-0.017631,-0.018442,...,1364.178894,1576.224406,1625.117301,1100.684447,0.197799,10.543943,10.556362,10.771629,484379.0,0.005427
524559,0.000216,0.014031,0.003584,-0.001199,-0.008458,-5.796803e-03,-0.004567,0.005819,-0.002661,-0.003891,...,1991.979684,1418.626035,1253.361174,1726.009481,0.255014,8.605063,8.594749,8.726646,882447.0,0.022712
525278,-0.009049,0.006995,-0.003221,-0.005797,0.006212,-1.123333e-08,-0.007774,-0.000493,0.006212,0.013987,...,1544.120471,1322.711271,1227.455657,1456.009146,0.198565,20.052948,19.937037,20.193700,305566.0,-0.001233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474687,-0.003156,-0.002843,-0.002389,-0.000601,0.000601,1.659979e-03,0.001832,0.004811,-0.001059,-0.001230,...,755.496437,859.870155,1097.978782,2185.348603,0.124384,109.489080,109.423294,109.178300,69860.0,0.005219
620200,-0.005474,-0.005335,-0.005509,-0.002454,0.000700,2.318624e-03,-0.000727,0.000411,-0.001619,0.001427,...,1072.835540,1005.341016,1437.991572,2256.730451,0.097487,85.336110,85.126970,84.881970,139268.0,0.001534
11908,0.002591,0.004417,0.002889,0.005610,0.001834,5.823757e-03,0.004319,0.003771,-0.003990,-0.002486,...,3146.915786,3221.191763,2999.656179,3712.759845,0.159906,47.260170,47.526077,47.469370,139122.0,0.001472
719403,0.009592,0.006035,0.002739,0.002199,-0.011761,5.547418e-03,0.009233,-0.000683,-0.017308,-0.020993,...,882.373592,592.626950,938.420333,1328.093235,0.174713,12.221778,12.248676,12.295748,379523.0,0.001981


In [ ]:
mwd

In [ ]:
mwd.to_csv("mvd.csv", index = False)

# Feature selection

## VIF

In [ ]:
data_scaled = normalize(m)
data_scaled = pd.DataFrame(data_scaled, columns = m.columns)

In [ ]:
data_scaled.var()

In [ ]:
# feature hotmap captures only the pairwise correlation between the features
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = m.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
def calculateVIF(X):
    vif_data = pd.DataFrame({
        "Names": X.columns,
        "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    })
    return vif_data

In [ ]:
vif = calculateVIF(m)
while(vif["VIF"].max() >= 10):
    drop_var = vif[vif.VIF == vif["VIF"].max()]["Names"]
    m = m.drop(drop_var, axis = 1)
    vif = calculateVIF(m)

In [4]:
keep_var = ['10/17res', '10/17raw', '16/17res', '16/17raw', '17ma3raw', '17ma3res',
       '17ma5res', 'market_value', '10/17vol', '16/17vol', 'turnover_ma1',
       'turnover_ma20', 'estVol', 'cleanMid17', 'vol17', 'y']
m = m.loc[:,keep_var]

In [5]:
m.columns.shape

(16,)

In [6]:
m.columns

Index(['10/17res', '10/17raw', '16/17res', '16/17raw', '17ma3raw', '17ma3res',
       '17ma5res', 'market_value', '10/17vol', '16/17vol', 'turnover_ma1',
       'turnover_ma20', 'estVol', 'cleanMid17', 'vol17', 'y'],
      dtype='object')

# Train-valid-test split

In [7]:
# train, test data split
train_valid = m.loc[list(mwd[mwd["Date"] < 20170101].index)] # 2014-2016
test = m.loc[list(mwd[mwd["Date"] >= 20170101].index)] # 2017

X_train_valid, y_train_valid = train_valid.iloc[: , :-1], train_valid["y"]
X_test, y_test = test.iloc[: , :-1], test["y"]

# scale the features in train set
sc = StandardScaler()
X_train_valid_scaled = sc.fit_transform(X_train_valid)
X_train_valid_scaled = pd.DataFrame(data = X_train_valid_scaled, columns = X_train_valid.columns, index = X_train_valid.index)

# do the same scaling to test set
X_test_scaled = sc.transform(X_test)
X_test_scaled = pd.DataFrame(data = X_test_scaled, columns = X_test.columns)

X_train = X_train_valid.loc[list(mwd[mwd["Date"] < 20160101].index)] # 2014-2015
X_valid = X_train_valid.loc[list(mwd[(mwd["Date"] >= 20160101) & (mwd["Date"] < 20170101)].index)] # 2016
y_train = m.loc[list(mwd[mwd["Date"] < 20160101].index)]["y"]
y_valid = m.loc[list(mwd[(mwd["Date"] >= 20160101) & (mwd["Date"] < 20170101)].index)]["y"]


# X_train_scaled = X_train_valid_scaled.loc[list(mwd[mwd["Date"] < 20160101].index)] # 2014-2015
# X_valid_scaled = X_train_valid_scaled.loc[list(mwd[(mwd["Date"] >= 20160101) & (mwd["Date"] < 20170101)].index)] # 2016
# y_train = m.loc[list(mwd[mwd["Date"] < 20160101].index)]["y"]
# y_valid = m.loc[list(mwd[(mwd["Date"] >= 20160101) & (mwd["Date"] < 20170101)].index)]["y"]

# X_train_valid_scaled = X_train_valid_scaled.reset_index(drop=True)
# X_train_scaled = X_train_scaled.reset_index(drop=True)
# X_valid_scaled = X_valid_scaled.reset_index(drop=True)

In [8]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_train_valid.shape, y_train_valid.shape, X_test.shape, y_test.shape

((378660, 15),
 (378660,),
 (195707, 15),
 (195707,),
 (574367, 15),
 (574367,),
 (194715, 15),
 (194715,))

# Cross validation

In [9]:
def weighted_mse(model, y_true, X):
    r2_weight = 1 / np.array(X['estVol'])
    r2_weight = r2_weight / r2_weight.sum()
    y_pred = model.predict(X)
    return r2_score(y_true, y_pred, sample_weight = r2_weight)

def cv_evaluate(model, X, y, how, cv_folds = 5):
    # cross validation
    cv_train_r2, cv_valid_r2 = [], []    
    
    if how == "walk_forwarding":
        tscv = TimeSeriesSplit(n_splits = cv_folds)
        splits = tscv.split(X)
        for train_index, test_index in splits:
            X_train_cv, X_valid_cv = X.iloc[train_index], X.iloc[test_index]
            y_train_cv, y_valid_cv = y.iloc[train_index], y.iloc[test_index]

            model.fit(X_train_cv, y_train_cv)
            cv_train_r2 += [weighted_mse(model, y_train_cv, X_train_cv)]
            cv_valid_r2 += [weighted_mse(model, y_valid_cv, X_valid_cv)]
    else:
        n_samples = len(X)
        folds = n_samples // cv_folds
        indices = np.arange(n_samples)

        margin = 0
        for i in range(cv_folds): 
            start = i * folds  
            stop = start + folds  
            temp = int(0.8 * (stop - start)) + start #If you want to change the data ratio of train/Validation, change the 0.8 part.
        
            X_train_cv, X_valid_cv = X.iloc[start: temp], X.iloc[temp + margin: stop]
            y_train_cv, y_valid_cv = y.iloc[start: temp], y.iloc[temp + margin: stop]

            model.fit(X_train_cv, y_train_cv)
            cv_train_r2 += [weighted_mse(model, y_train_cv, X_train_cv)]
            cv_valid_r2 += [weighted_mse(model, y_valid_cv, X_valid_cv)]
    
    print("Cross Validation Report")
    print(f"cv train r2: {np.array(cv_train_r2).mean()*100}%, cv valid r2: {np.array(cv_valid_r2).mean()*100}%")
    
    return np.mean(cv_valid_r2)  

In [10]:
def parameter_tuning(my_model, X, y, X_valid, y_valid, X_train_valid, y_train_valid, X_test, y_test, para_test, cv_folds = 5, how = "walk_forwarding"):
    
    # cross validation to select best variables
    keys, values = zip(*para_test.items())
    permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
    weighted_r2_max = -float("inf")
    for p in permutations_dicts:
        print('\n',p)
        my_model.set_params(**p)
        weighted_r2 = cv_evaluate(my_model, X_train_valid, y_train_valid, how, cv_folds)
        if weighted_r2 > weighted_r2_max:
            weighted_r2_max = weighted_r2
            best_paras = p
    my_model.set_params(**best_paras)  
    print(f"best_paras: {best_paras}")
    
    return best_paras

In [11]:
def extratree_fit(model, X_train, y_train, X_test, y_test):
    
    # Fit the algorithm on the data
    model.fit(X_train, y_train)         

    # calculate weighted r2
    weight_r2_train = weighted_mse(model, y_train, X_train)
    weight_r2_test = weighted_mse(model, y_test, X_test)

    #Print model report:
    print("\nModel Report")
    print(f"train r2: {weight_r2_train*100}%, test r2: {weight_r2_test*100}%")
    return model

## parameter selection

### walk_forwarding

In [12]:
permutations_extra = {'n_estimators': [50, 100, 150],
                      'min_samples_leaf': [3,5],
                      'min_samples_split': [2,4],
                      'criterion': ['mse'],
                      'max_depth': [8,32]}  

In [13]:
# permutations_extra = {'n_estimators': [50],
#         'min_samples_leaf': [30],
#         'min_samples_split': [25]}  

In [ ]:
my_model = ExtraTreesRegressor()
best_paras = parameter_tuning(my_model, X_train, y_train, 
                              X_valid, y_valid, X_train_valid, y_train_valid, 
                              X_test, y_test, permutations_extra, cv_folds = 5, how = "walk_forwarding")


 {'n_estimators': 50, 'min_samples_leaf': 3, 'min_samples_split': 2, 'criterion': 'mse', 'max_depth': 8}
Cross Validation Report
cv train r2: 0.917625135897151%, cv valid r2: 0.11839424321462122%

 {'n_estimators': 50, 'min_samples_leaf': 3, 'min_samples_split': 2, 'criterion': 'mse', 'max_depth': 32}
Cross Validation Report
cv train r2: 47.50863512220659%, cv valid r2: -0.7661144266863085%

 {'n_estimators': 50, 'min_samples_leaf': 3, 'min_samples_split': 4, 'criterion': 'mse', 'max_depth': 8}
Cross Validation Report
cv train r2: 0.9339834643993328%, cv valid r2: 0.12889945387197788%

 {'n_estimators': 50, 'min_samples_leaf': 3, 'min_samples_split': 4, 'criterion': 'mse', 'max_depth': 32}
Cross Validation Report
cv train r2: 47.23347201302241%, cv valid r2: -0.7559577089146297%

 {'n_estimators': 50, 'min_samples_leaf': 5, 'min_samples_split': 2, 'criterion': 'mse', 'max_depth': 8}
Cross Validation Report
cv train r2: 0.8731231288146035%, cv valid r2: 0.14310841854051626%

 {'n_estim

In [ ]:
my_model.set_params(**best_paras)  
my_model = extratree_fit(my_model, X_train_valid, y_train_valid, X_test, y_test)

### rolling

In [ ]:
my_model = ExtraTreesRegressor()
best_paras = parameter_tuning(my_model, X_train, y_train, 
                              X_valid, y_valid, X_train_valid, y_train_valid, 
                              X_test, y_test, permutations_extra, cv_folds = 5, how = "rolling")

In [ ]:
my_model.set_params(**best_paras)  
my_model = extratree_fit(my_model, X_train_valid, y_train_valid, X_test, y_test)

## Permutation feature importance

##  eli5

In [ ]:
perm = PermutationImportance(my_model)
perm.fit(X_train_valid, y_train_valid)

In [ ]:
eli5.show_weights(perm, feature_names = X_train_valid.columns.tolist())

In [ ]:
# perm.feature_importances_ attribute is now available, it can be used
# for feature selection - let's e.g. select features which increase
# accuracy by at least 0.05:
#sel = SelectFromModel(perm, threshold=0.001, prefit=True)
#X_train_valid_new = pd.DataFrame(sel.transform(X_train_valid))
#X_train_valid_new

# Model fit, summary

In [ ]:
best_paras = { 'max_depth': 4,
                    'min_child_weight':5,
                    'n_estimators': 100,
                    'gamma': 0,
                    'learning_rate': 0.1} 

In [ ]:
my_model = ExtraTreesRegressor()
my_model.set_params(**best_paras)  
my_model = extratree_fit(my_model, X_train_valid, y_train_valid, X_test, y_test)

## sklearn inspection

In [ ]:
result = permutation_importance(
    my_model, X_train_valid, y_train_valid)

sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=X_train_valid.columns[sorted_idx]
)
ax.set_title("Permutation Importances (training set)")
fig.tight_layout()
plt.show()

## get_score

In [ ]:
feature_important = my_model.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.nlargest(40, columns="score").plot(kind='bar', title='Feature Importances', figsize = (15,10)) ## plot top 40 features
plt.ylabel('Feature Importance Score')

# Plots

# binplot

###  y vs y_pred

In [ ]:
y_pred = my_model.predict(X_test)  
ans = pd.DataFrame.from_dict({'y_pred': y_pred, 'y': y_test})
ans.to_csv('predictions.csv',index=False)    

In [ ]:
def bin_plot(df, x_label, y_label, n_bins, path, w_label=None, scale=1, ax=None, **plot_kwargs):
        
        '''
        create a bin plot with your feature (x_label), binned into (num_bin) quantiles 
        (with equal count), on the X-axis, and the corresponding mean of the target
        variable (y_label) along with one standard error around mean on the Y-axis. 
        
        Parameters
        ----------------
        x_label: str
            label of feature x
            
        y_label: str
            label of target
            
        w_label: str
            label of weight if provided
            
        scale: float, default=1
            where to plot the point with the bin = mean(bin) * scale
            
        ax: axes
            ax to plot on, if None, the function will create one
            
        n_bins: int
            number of bins
            
        **plot_kwargs: dict
            keyword and arguments of ax.plot(x, y)
        '''
        
        key = df[x_label].values
        value = df[y_label].values
        if w_label is not None:
            w = df[w_label].values
        
            
        sorted_idx = np.argsort(key)
        chunks = np.array_split(sorted_idx, n_bins)
              
        # x axis on the plot: bin mean
        x = np.ones(n_bins)
        # boundary of the bins
        b = np.ones(n_bins)
        # y axis on the plot: bin mean
        y = np.ones(n_bins)
        # confidence of y: bin std
        c = np.ones(n_bins)
        
        for bin_idx, idx in enumerate(chunks):
            
            x[bin_idx] = np.mean(key[idx]) * scale
            b[bin_idx] = np.max(key[idx])
            if w_label is not None:
                y[bin_idx] = np.average(value[idx], weights=w[idx])
            else:
                y[bin_idx] = np.mean(value[idx])
            c[bin_idx] = np.std(value[idx]) / np.sqrt(len(value[idx]))
        
        if ax is None: 
            fig, ax = plt.subplots(figsize=(10, 6))
            
        ymin = np.min(y-c) - 0.5 * abs(np.min(y-c))
        ymax = np.max(y+c) + 0.5 * abs(np.max(y-c))
            
        ax.plot(x, y, 'o-', **plot_kwargs)
        ax.fill_between(x, y-c, y+c, color='b', alpha=0.1)
        ax.vlines(b[:-1], ymin=ymin, ymax=ymax, color='grey')
        ax.set_xticks(b[:-1])
        ax.set_ylim(bottom=ymin, top=ymax)
        ax.set_xlabel('bin boundaries, based on quantiles of {:s}'.format(x_label))
        ax.set_ylabel('within-bin means and 1 std error of {:s}'.format(y_label))
        ax.set_title('{:d} binned plot of {:s} against {:s}'.format(n_bins, y_label, x_label))
        plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right') 
        fig.savefig(path+'bin_plot_'+x_label.replace('/','_')+'.png')

In [ ]:
path = r'C:\\Users\\Lenovo\\Desktop\\2022Spring\\9899 Machine learning\\project\\latex\\'
bin_plot(ans, 'y_pred', 'y', 10, path, w_label=None, scale=1)

### y vs feature

In [ ]:
for col in m.columns:
    bin_plot(m, col, 'y', 10, path, w_label=None, scale=1)    

## Drift plot